# Welcome

Welcome to this interactive notebook showcasing the new features of C# 13. In this notebook, we will explore some of the exciting enhancements and additions to the C# language.
You will find code examples and explanations that demonstrate how these new features can be used to write more efficient and expressive code. Let's dive in and see what's new in C# 13!
This showcase will follow the [official post](https://learn.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-13) about the newest features in C#, if you want further information, you will find them there!

## Intro
Before we dive into the new features, lets get familiar with Polyglot notebooks. If your are familiar with Jupiter notebooks, you can probably skip this part. For all new to notebooks, let me explain the basics:
In a notebook, you can tell an interactive journey using text and code. Below you will find your first code cell, update the variable `name`, to hold your name.

In [2]:
var name = "Bjarne";

As you can see, the variable is not scoped to a single cell. Instead we can use it in the cell above to welcome you.
Congrats, you just solved your first cell! Now check the output below. 

In [ ]:
Console.WriteLine("Hello, " + name + "!");

# Params Collection
You probably heard of them and the `params` keyword, but rarely used it: Params collections. Let's refresh our memories and fix the code using params collection.

In [ ]:
public void PrintStrings(string[] strings)
{
    foreach (var s in strings)
    {
        Console.WriteLine(s);
    }
}

PrintStrings("Donald", "Mickey", "Goofy");

The `params` modifier allows us to create a method that takes an arbitrary number of parameters of the same type wihthout using an array to call the method. In our example, we can just pass three strings.
In C# 12, `params` can only be applied to arrays, not `ImmutableArray<T>` or even `Span<T>`. This changed in C#13 where we can use params for any valid collection epxression type such as `Span<T>`, `ICollection<T>`, `IEnumerable` and much more.
 

In [ ]:
public void PrintStrings(params ReadOnlySpan<string> strings)
{
    foreach (var s in strings)
    {
        Console.WriteLine(s);
    }
}

PrintStrings("Donald", "Mickey", "Goofy");  

# New Lock Object
If you work with multiple Threads / Taks in C#, you may encounter the situation in which a special part of your logic should not be executed in parallel.
For example a dictionary, one should not delete and access the same key at the same point in time. Instead we want execute the different actions after each other and not in parallel. This can be achieved with the `lock(x){}` statement:

In [ ]:
var lockObject = new Object();

lock(lockObject)
{
    Console.WriteLine("Only one thread at a time.");
}

Up to C# 12, any `Object` could be used as a `lockObject`. When a given Thread entered the `lock` block, it would use the `lockObject` to mark that is has entered the block, preventing other threads from entering the block with the same object.
Internally, the compiler would transpile the `lock`statement into:
```C#
object __lockObj = x;
bool __lockWasTaken = false;
try
{
    System.Threading.Monitor.Enter(__lockObj, ref __lockWasTaken);
    Console.WriteLine("Only one thread at a time.");
}
finally
{
    if (__lockWasTaken) System.Threading.Monitor.Exit(__lockObj);
}

```
Using `System.Threading.Monitor.Enter` for handling the different threads, blocking them until the previous Thread called `Exit`.


C# 13 introduces a new special Type `System.Threading.Lock` to be used as the `lockObject`: 

In [ ]:
var lockObject = new System.Threading.Lock();

lock(lockObject)
{
    Console.WriteLine("Only one thread at a time.");
}

If the lock statement is used with the new Type, the compiler generates a different result, based on a disposbale scope acquired by the lock using the familiar `IDisposable` pattern. If the scope is disposed, another thread can enter the locked scoped. Allowing a more streamline approach at thrad synchronization based on known patterns.

In [ ]:
using (lockObject.EnterScope())
{
    Console.WriteLine("Only one thread at a time.");
}

# New Escape Sequence 
C# 13 introduces a new character literal. While character literals may not ring a bell, you are probably familiar with the character literal `\n` representing a new line. In the newest version of C#, you can use `\e` as the escape sequence for the `ESCAPE` character, which was previously achieved via `\u001b`.
The `ESCAPE`character allows the usage of the old [ANSI-Escape-Codes](https://en.wikipedia.org/wiki/ANSI_escape_code), allowing different text formatting:

In [ ]:
Console.WriteLine("Hello, in regular Text");
Console.WriteLine("\u001b[1mHello, in bold Text\u001b[0m");
Console.WriteLine("\u001b[4mHello, in underlined text\u001b[0m");

With the new character literal `\e`, these become easier to use:

In [ ]:
Console.WriteLine("Hello, in regular Text");
Console.WriteLine("\e[1mHello, in bold Text\e[0m");
Console.WriteLine("\e[4mHello, in underlined text\e[0m");

# Implicit Index Access
C# 8 introduced the two types `System.Index` and `System.Range` allowing easy indexing and enumration of collections:

In [10]:
// Helper function that prints a given array.
public void PrintArray<T>(string title, T[] array)
{
    Console.Write($"{title}: ");
    foreach (var item in array)
    {
        Console.Write($"{item}, ");
    }
    Console.WriteLine();
}

In [ ]:
int[] numbers = [1,2,3,4,5,6,7,8,9,10];

var myNumbers = numbers[2..5];
PrintArray(nameof(myNumbers), myNumbers);

var lastTwo = numbers[^2..];
PrintArray(nameof(lastTwo), lastTwo);

In C# 13, we can now use the *from the end* index operator `^`, for object initialization

In [ ]:
var coolNames = new string[4];

coolNames = {
    [^1] = "Mickey",
    [^2] = "Donald",
    [^3] = "Goofy",
    [^4] = "Pluto"
};
PrintArray(nameof(coolNames), coolNames);

# Use `ref` and `unsafe` in iterators and `async` methods
If you are unfamiliar with `ref, unsafe` and `ref struct`, I recommend reading [this explanaition](https://okyrylchuk.dev/blog/understanding-struct-vs-ref-struct-in-csharp/) from Oleg Kyrylchuk.
As a short summary, while a `struct` type can be boxed and thus, moved to the heap, a `ref struct` can not be moved to the heap.

C# 13 allows the usage of `ref` and `unsafe` within iterators and async methods, as long as they stay within the `await` boundary or do not cross a `yield return`.
For example, one is now able to define or use a `Span<T>` which is a `ref struct` inside an `async` method.

In [ ]:
public async Task<bool> MyAsyncMethod()
{
    await Task.Delay(1000);
    var span = "Hello, World!".AsSpan();
    var result = span.EndsWith("World");
    return result;
}

# `ref struct` interfaces

As stated before, some constraints regarding `ref` and `unsafe` where lifted. In C# 13, `ref structs` are now able to implement interfaces. In contrast to types which can be moved to heap, such `ref structs`can not be cast to the interface, as this would box them and thus, move them to the heap. This feature is especially useful in combination with the next lifted constraint regarding generics.

In [ ]:
interface IMyInterface
{
    void MyMethod();
}

ref struct MyRefStruct : IMyInterface
{
    public void MyMethod()
    {
        Console.WriteLine("MyMethod called.");
    }
}

void TestMyRefStruct()
{
    var myRefStruct = new MyRefStruct();
    myRefStruct.MyMethod();

    // Not allowed, as ref structs cannot be boxed
    IMyInterface myInterface = myRefStruct;
    
}


# New generic *anti-constraint* `allows ref struct`
With C# 13, `ref struct` can be passed into a generic with the new *anti-constraint* `allows ref struct`. It's called an *anti-constraint* becuase in constrat to existing constraints, it is not limiting but widening the possible used types.
With this new feature, `ref struct` types can be used with generics as seen in the example below. This is especially useful in combinatin with the new feature above, allowing `ref struct` to implement interfaces.

The example below illustrates a performant implementation of the `IHeadElement<T>` interface with a `ref struct` that is used as a type argument for the `HeadPrinter`. 
While this is not an useful example, the underlying principle could be used, to create an efficient, `Span<T>`based implementation of various interfaces such as `IReadOnlyList<T>` for example.

In [ ]:

public interface IHeadElement<T>{    
    T Head { get; }

}

public ref struct MyRefStruct : IHeadElement<int>{
    public int Head { get; set; }
}


public class HeadPrinter<T, T2> where T : IHeadElement<T2> , allows ref struct
{
    public void Print(scoped T refStruct) { 
        Console.WriteLine(refStruct.Head);
    }
}



public void Test() {
    var headPrinter = new HeadPrinter<MyRefStruct, int>();

    MyRefStruct myRef = new MyRefStruct();
    myRef.Head = 42;

    headPrinter.Print(myRef);

}

Test();

# More `partial` Members
The existing `partial` keyword allows the separation of a *declaring* and *implementing* definition of members such as methods. This is often used in combination with [Source Generators](https://learn.microsoft.com/en-us/dotnet/csharp/roslyn-sdk/#source-generators) with the `GeneratedRegex` attribute being a prominent example:

```csharp
using System.Text.RegularExpressions;

public partial class RegExSourceGenerator
{
    [GeneratedRegex("hen|rooster", RegexOptions.IgnoreCase, "en-US")]
    private static partial Regex HenOrRosterRegex();

    private static void EvaluateText(string text)
    {
        if (HenOrRosterRegex().IsMatch(text))
        {
            Console.WriteLine("Found a Hen or Rooster!");
        }
        else
        {
            Console.WriteLine("No Hen or Rooster found.");
        }
    }
}

```
In this example, a partial Member `HenOrRosterRegex` is defined. Its implementation is generated by the [Regular expression source generator](https://learn.microsoft.com/en-us/dotnet/standard/base-types/regular-expression-source-generators).

With C# 13, the `partial` keyword can now be applied to properties and indexers.

In [ ]:
partial class PartialClass
{
    public partial string MyPartialProperty {get;set;}

    public partial char this[int index] {get;}
}

// This can be in its own file
partial class PartialClass
{
    private string _myPartialProperty;
    public partial string MyPartialProperty
    {
        get => _myPartialProperty;
        set => _myPartialProperty = value;
    }

    public partial char this[int index]
    {
        get => _myPartialProperty[index];
    }
}

var partialClass = new PartialClass();

partialClass.MyPartialProperty = "Hello, World!";

Console.WriteLine(partialClass.MyPartialProperty);
Console.WriteLine(partialClass[1]);

# Overload resolution priority
C# 13 introduces a new `OverloadResolutionPriority` attribute. With this attribute, one can define which overload is the prefered one, when two ambiguous overloads are available. The constructor takes an integer defining the priority as an argument. Overloads without the attribute have a default priority of zero. This can be useful for libary authors which introduce a new, prefered overload.


Feel free to remove the `OverloadResolutionPriority` attribute in the example below and see the effect!

In [ ]:
using System.Runtime.CompilerServices;

public class OverloadResolutionPriorityExample{
    [OverloadResolutionPriority(1)]
    public void PrintString(params ReadOnlySpan<string> s) => Console.WriteLine("Span Overload");

    public void PrintString(params string[] strings) => Console.WriteLine("Array based");

}

var stringPrinter = new OverloadResolutionPriorityExample();

string[] strings = {"Hello", "World"};

stringPrinter.PrintString(strings);

stringPrinter.PrintString("Hello, World!");

# Introducing the `field` Keyword
This new feature is still in preview in C#13 and can be enabled by setting the `<LangVersion>` to `preview` within your project file. Enabling this feature introduces a new token `field`, which may introduce a breaking change in you codebase, as `field` is now a reserved token of C#. As a mitigashiation, you can rename your usage of `field` into `@field` or `this.field`.

The new keyword can be used in getters and setters to access the compiler synthesized backing field of a property and thus reduce boilerplate code:

In [ ]:
class Ticket {
    public int Price {
        get;
        set => field = (value > 0) 
            ? value 
            : throw new ArgumentOutOfRangeException("Price must be positive");;
    }
}